In [1]:
import os
cache_dir = '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME']=cache_dir
os.environ['HF_HUB_CACHE']=cache_dir+'/hub'
hf_token=os.getenv('hf_token')

In [2]:
import re
import math
import torch
import numpy as np
import random
from datasets import load_from_disk, Dataset, load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM

/work/pi_mccallum_umass_edu/kchimmad_umass_edu/conda_envs/reason/lib/python3.12/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


### Preparing data in preference format


In [27]:
# data_path='../datasets/gsm8k/dpo/dpo_data_tr_prob_threshold'
data_path='../datasets/neutralization/dpo/LLaMA3B/feedback-100/dpo_data_with_teacher_gen'
dpo_data=load_from_disk(data_path)
dpo_data

Dataset({
    features: ['prompt', 'chosen', 'rejected', 'logprob_ratio', 'tr_prob', 'tr_answer', 'stu_answer', 'tr_score'],
    num_rows: 100
})

In [28]:
print(dpo_data['prompt'][0])

<|start_header_id|>system<|end_header_id|>

You are an expert in removing subjective biases in texts .<|eot_id|><|start_header_id|>user<|end_header_id|>

Instruction:
Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:
1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);
2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;
3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).

Your response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the input sentence.

Input:
flashpacking refers to an affluent backpacker.<|eot_id|><|start_header_id|>assi

In [29]:
print(dpo_data['chosen'][0])

To neutralize the sentence, we need to address potential biases. The term "affluent" might be seen as having a slightly positive connotation, implying a certain level of respect or admiration for the individual's financial status. However, the primary subjective element here is the term "affluent" itself, which can be perceived as having a positive bias due to its association with wealth.

A more neutral approach would be to simply state the fact without implying a value judgment on the financial status. Thus, instead of "affluent," we could use a term that simply describes the situation without the positive connotation, such as "well-funded" or, more simply, describe the behavior or characteristics without focusing on the financial aspect.

The neutralized text is: flashpacking refers to a backpacker with a high budget.


In [30]:
print(dpo_data['rejected'][0])

To neutralize the subjective bias in the given sentence, we need to replace the word "flashpacking" with a more neutral term.

"Flashpacking" is a term that is often associated with a particular point of view, implying that it is a luxurious or indulgent form of backpacking. To replace it with a neutral term, we can use the word "backpacking".

The neutralized text is Backpacking refers to an affluent traveler.


In [21]:
import json
def formatting_prompts_func_gec(example):
    with open('../prompts/gec.json') as fp:
        task_prompt = json.load(fp)
    system_msg= f'<|start_header_id|>system<|end_header_id|>\n\n{task_prompt['system_msg']}<|eot_id|>'
    user_msg= f'<|start_header_id|>user<|end_header_id|>\n\n{task_prompt['user_msg'].format(instruction=task_prompt['task_prompt'], question=example['input'])}<|eot_id|>'
    rationale= example['rationale'] if 'rationale' in example else '' # vanilla sft using off-the-shelf data doesn't have rationale
    # For GEC multiple editors are allowed, so when we use off-the-shelf data we need to choose one of the output or else train it as n different examples
    # Using to determine off-the-shelf data or custom data
    if 'rationale' in example:
        assistant_msg=f'<|start_header_id|>assistant<|end_header_id|>\n\n{task_prompt['assistant_msg'].format(response=example['output'], rationale=rationale)}<|eot_id|>' 
    else: # Currently taking first output for off-the-shelf data
        assistant_msg=f'<|start_header_id|>assistant<|end_header_id|>\n\n{task_prompt['assistant_msg'].format(response=example['output'][0], rationale=rationale)}<|eot_id|>' 
    text= system_msg + user_msg + assistant_msg
    return text

In [9]:
data=load_from_disk('../datasets/gec/raw/feedback-100')
data[0]

{'id': '9834',
 'input': 'However , it has a nice and three bathrooms .',
 'edits': ['A 0 1|||R:OTHER|||In addition|||REQUIRED|||-NONE-|||0',
  'A 5 6|||UNK|||nice|||REQUIRED|||-NONE-|||0'],
 'output': ['In addition , it has a nice and three bathrooms .']}

## DPO Code

In [5]:
# train_dpo.py
from trl import DPOConfig, DPOTrainer
# from trl import DataCollatorForPreference
from transformers import AutoModelForCausalLM, AutoTokenizer
from small_llm_reasoning.trainer.dpo_trainer import CustomDPOTrainer


In [6]:
model_name='meta-llama/Llama-3.2-3B-Instruct'
output_dir= 'dpo_data_tr_stu_correctness_2'
add_special_tokens= True
epochs= 5
lr=1e-5 
lr_scheduler_type= 'cosine'
warmup= 0.1 
weight_decay= 0.01
per_device_train_batch_size= 4
gradient_accumulation_steps= 4
max_length= 500 
torch_dtype='bfloat16'

In [7]:
# Load tokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=hf_token, 
    cache_dir=cache_dir
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.add_special_tokens=add_special_tokens

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    torch_dtype=torch_dtype,
    token=hf_token, 
    cache_dir=cache_dir
)

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-3B-Instruct/.no_exist/0cb88a4f764b7a12671c53f0838cd831a0843b95/adapter_config.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-3B-Instruct/.no_exist/0cb88a4f764b7a12671c53f0838cd831a0843b95/adapter_config.json'
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.57it/s]


In [8]:
# math.ceil((dpo_data.num_rows*epochs)/(per_device_train_batch_size*gradient_accumulation_steps))

In [9]:
# Training arguments
training_args = DPOConfig(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=epochs,
    learning_rate=lr, # Default 1e-6
    lr_scheduler_type=lr_scheduler_type,        
    weight_decay=weight_decay,
    save_strategy="epoch",
    warmup_ratio=warmup,
    logging_steps=10,
    dataloader_drop_last=False,
    # max_steps=math.ceil((dpo_data.num_rows*epochs)/(per_device_train_batch_size*gradient_accumulation_steps)),
    max_length=max_length
)

In [10]:
# # Initialize trainer
# trainer = DPOTrainer(
#     model=model, 
#     args=training_args, 
#     processing_class=tokenizer, 
#     train_dataset=dpo_data)



In [11]:
trainer = CustomDPOTrainer(
    model=model, 
    args=training_args, 
    processing_class=tokenizer, 
    train_dataset=dpo_data)

In [12]:
output=trainer.train()

num_train_epochs: 5
num_update_steps_per_epoch: 4
num_examples: 71
num_train_samples: 355
epoch_based: True
len_dataloader: 18
max_steps: 20
callback: [<transformers.trainer_callback.DefaultFlowCallback object at 0x71fff7809fa0>, <transformers.utils.notebook.NotebookProgressCallback object at 0x71fff7bdad80>]
Total_updates: 5
Update-Step: 0
before on_step_end: [<transformers.trainer_callback.DefaultFlowCallback object at 0x71fff7809fa0>, <transformers.utils.notebook.NotebookProgressCallback object at 0x71fff7bdad80>]


Step,Training Loss


after on_step end: [<transformers.trainer_callback.DefaultFlowCallback object at 0x71fff7809fa0>, <transformers.utils.notebook.NotebookProgressCallback object at 0x71fff7bdad80>]


NameError: name 'kedar' is not defined

In [ ]:
def get_log_prob_ratio(teacher_log_prob, student_log_prob):
    tr_stu_logprob=[]
    student_logprob=[]
    teacher_logprob=[]
    for i in range(len(teacher_log_prob)):
        student_log_probs=np.array(student_log_prob[i])
        teacher_log_probs=np.array(teacher_log_prob[i])
        student_logprob.append(np.mean(student_log_probs))
        teacher_logprob.append(np.mean(teacher_log_probs))
        tr_stu_logprob.append(
            np.subtract(
                np.mean(teacher_log_probs),
                np.mean(student_log_probs)
            )
        )
                
    return tr_stu_logprob
def merge_and_sample_data(data,teacher_data,student_data, remove_incorrects, sampling_ratio, seed=42 ):
    # print(teacher_data['log_probs'][1])
    # print(len(teacher_data['log_probs']))
    tr_stu_logprob_ratio=get_log_prob_ratio(teacher_data['log_probs'],student_data['student_log_probs'])
    threshold=np.median(tr_stu_logprob_ratio)
    teacher_answers=[]
    teacher_scores=[]
    print(f'Median teacher-student-logprob-ratio: {threshold}')
    for i in range(teacher_data.num_rows):
        teacher_answers.append(teacher_data['output'][i][0])
        teacher_scores.append(teacher_data['score'][i])
    questions=data['question']
    new_data = {
        'question': questions,
        'answer': teacher_answers,
        'score': teacher_scores,
        'logprob_ratio':tr_stu_logprob_ratio
    }
    
    data= Dataset.from_dict(new_data)

    if remove_incorrects:
        data= data.filter(lambda x: x['score']==1)
    print(f'After removing incorrects from teacher:{data.num_rows}')
    
    
    rng = random.Random(seed)

    total_size = len(data)
    size_below = int(total_size * sampling_ratio)
    size_above = total_size - size_below

    below_thresh = data.filter(lambda example: example['logprob_ratio'] < threshold)
    above_thresh = data.filter(lambda example: example['logprob_ratio'] >= threshold)
    print(f'below threshold:{below_thresh.num_rows}')
    print(f'above threshold:{above_thresh.num_rows}')

    def upsample(ds, target_size):
        if len(ds) == 0:
            return ds  # Avoid divide-by-zero
        indices = [rng.randint(0, len(ds) - 1) for _ in range(target_size)]
        return ds.select(indices)

    sampled_below = upsample(below_thresh, size_below)
    sampled_above = upsample(above_thresh, size_above)

    data = concatenate_datasets([sampled_below, sampled_above])
    return data.shuffle(seed=seed)
    
    
    
    

In [4]:
import re
import random
import numpy as np
from datasets import load_from_disk, Dataset, load_dataset, concatenate_datasets


In [17]:
data_path='../datasets/gsm8k/dpo/dpo_data_with_teacher_gen'
data=load_from_disk(data_path)
data

Dataset({
    features: ['prompt', 'chosen', 'rejected', 'tr_prob', 'tr_score', 'tr=stu'],
    num_rows: 1000
})

In [11]:
sampling_col='tr_prob'
sampling_ratio=0.9
seed=42
threshold=0.6

In [12]:
rng = random.Random(seed)

total_size = len(data)
size_below = int(total_size * sampling_ratio)
size_above = total_size - size_below

In [13]:
below_thresh = data.filter(lambda example: example[sampling_col] < threshold)
above_thresh = data.filter(lambda example: example[sampling_col] >= threshold)
print(f'below threshold:{below_thresh.num_rows}')
print(f'above threshold:{above_thresh.num_rows}')

def upsample(ds, target_size):
    if len(ds) == 0:
        return ds  # Avoid divide-by-zero
    indices = [rng.randint(0, len(ds) - 1) for _ in range(target_size)]
    return ds.select(indices)

sampled_below = upsample(below_thresh, size_below)
sampled_above = upsample(above_thresh, size_above)

data = concatenate_datasets([sampled_below, sampled_above])
data=data.shuffle(seed=seed)
data

Filter: 100%|██████████| 1000/1000 [00:00<00:00, 133926.30 examples/s]

below threshold:93
above threshold:907


Dataset({
    features: ['prompt', 'chosen', 'rejected', 'tr_prob', 'tr_answer', 'stu_answer', 'tr_score'],
    num_rows: 1000
})

In [14]:
new_col=[]
for i in range(data.num_rows):
    new_col.append(data['tr_answer'][i]!=data['stu_answer'][i])

True

In [15]:
data = data.map(lambda example: {"tr!=stu": example["tr_answer"] != example["stu_answer"]})
data

Map: 100%|██████████| 1000/1000 [00:00<00:00, 13977.01 examples/s]


Dataset({
    features: ['prompt', 'chosen', 'rejected', 'tr_prob', 'tr_answer', 'stu_answer', 'tr_score', 'new_col'],
    num_rows: 1000
})

In [ ]:
def create_preference_data_with_teacher_gen_by_sampling(data_path, sampling_col, remove_incorrects=True)

In [1]:
import os
cache_dir = '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME']=cache_dir
os.environ['HF_HUB_CACHE']=cache_dir+'/hub'
hf_token=os.getenv('hf_token')

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

/work/pi_mccallum_umass_edu/kchimmad_umass_edu/conda_envs/reason/lib/python3.12/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
model_name='meta-llama/Llama-3.2-3B-Instruct'
tokenizer= AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)

chat=[
    {
        'role':'user',
        'content':'Hello World!'
    }
]


In [4]:
tokenizer.apply_chat_template(chat,tokenize=False)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 May 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHello World!<|eot_id|>'